In [9]:
import pandas as pd

In [17]:
bench_data = pd.read_csv('results.csv')

bench_data.head()

,Strategy,Ground Truth Binary,Ground Truth Source,Japicmp Binary,Japicmp Source,Revapi Binary,Revapi Source,Roseau Binary,Roseau Source
0,ReduceC3VisibilityToPackagePrivate,1,1,1,1,1,1,1,1
1,RemoveTypeC3,1,1,1,1,1,1,1,1
2,AddAbstractModifierToC3,1,1,1,1,1,1,1,1
3,AddFinalModifierToC3,1,1,1,1,1,1,1,1
4,RemoveFieldF25InApiC3,1,1,1,1,1,1,1,1


In [18]:
GROUND_TRUTH_KEY = 'Ground Truth'
tools_analyzed = ['Japicmp', 'Revapi', 'Roseau']

GENERAL_BREAKING_SUFFIX = ' Global'
BINARY_BREAKING_SUFFIX = ' Binary'
SOURCE_BREAKING_SUFFIX = ' Source'

full_data = pd.DataFrame(columns=['bench'])
full_data['bench'] = bench_data['Strategy']
full_data[f'{GROUND_TRUTH_KEY}{BINARY_BREAKING_SUFFIX}'] = bench_data[f'{GROUND_TRUTH_KEY}{BINARY_BREAKING_SUFFIX}'].map(lambda x: True if x == 1 else False)
full_data[f'{GROUND_TRUTH_KEY}{SOURCE_BREAKING_SUFFIX}'] = bench_data[f'{GROUND_TRUTH_KEY}{SOURCE_BREAKING_SUFFIX}'].map(lambda x: True if x == 1 else False)
full_data[f'{GROUND_TRUTH_KEY}{GENERAL_BREAKING_SUFFIX}'] = full_data[f'{GROUND_TRUTH_KEY}{BINARY_BREAKING_SUFFIX}'] | full_data[f'{GROUND_TRUTH_KEY}{SOURCE_BREAKING_SUFFIX}']

for tool in tools_analyzed:
    full_data[f'{tool}{BINARY_BREAKING_SUFFIX}'] = bench_data[f'{tool}{BINARY_BREAKING_SUFFIX}'].map(lambda x: True if x == 1 else False)
    full_data[f'{tool}{SOURCE_BREAKING_SUFFIX}'] = bench_data[f'{tool}{SOURCE_BREAKING_SUFFIX}'].map(lambda x: True if x == 1 else False)
    full_data[f'{tool}{GENERAL_BREAKING_SUFFIX}'] = full_data[f'{tool}{BINARY_BREAKING_SUFFIX}'] | full_data[f'{tool}{SOURCE_BREAKING_SUFFIX}']

full_data.head()

,bench,Ground Truth Binary,Ground Truth Source,Ground Truth Global,Japicmp Binary,Japicmp Source,Japicmp Global,Revapi Binary,Revapi Source,Revapi Global,Roseau Binary,Roseau Source,Roseau Global
0,ReduceC3VisibilityToPackagePrivate,True,True,True,True,True,True,True,True,True,True,True,True
1,RemoveTypeC3,True,True,True,True,True,True,True,True,True,True,True,True
2,AddAbstractModifierToC3,True,True,True,True,True,True,True,True,True,True,True,True
3,AddFinalModifierToC3,True,True,True,True,True,True,True,True,True,True,True,True
4,RemoveFieldF25InApiC3,True,True,True,True,True,True,True,True,True,True,True,True


In [19]:
def compute_metrics_for_breaking_type(data, suffix, tools):
    metrics_data = pd.DataFrame(columns=['tool', 'precision', 'recall', 'f1'])

    for tool in tools:
        tp = data[(data[f'{tool}{suffix}'] == True) & (data[f'{GROUND_TRUTH_KEY}{suffix}'] == True)].count().iloc[0]
        fp = data[(data[f'{tool}{suffix}'] == True) & (data[f'{GROUND_TRUTH_KEY}{suffix}'] == False)].count().iloc[0]
        fn = data[(data[f'{tool}{suffix}'] == False) & (data[f'{GROUND_TRUTH_KEY}{suffix}'] == True)].count().iloc[0]

        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * (precision * recall) / (precision + recall)

        metrics_data = pd.concat(
            [
                metrics_data,
                pd.DataFrame.from_records([{
                    'tool': tool,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1
                }])
            ],
            ignore_index=True
        )

    return metrics_data


In [20]:
general_breaking_metrics_data = compute_metrics_for_breaking_type(full_data, GENERAL_BREAKING_SUFFIX, tools_analyzed)

general_breaking_metrics_data

,tool,precision,recall,f1
0,Japicmp,0.563750,0.996154,0.720021
1,Revapi,0.552826,0.999794,0.711974
2,Roseau,0.994650,0.997964,0.996304


In [21]:
binary_breaking_metrics_data = compute_metrics_for_breaking_type(full_data, BINARY_BREAKING_SUFFIX, tools_analyzed)

binary_breaking_metrics_data

,tool,precision,recall,f1
0,Japicmp,0.537584,1.000000,0.699258
1,Revapi,0.537462,0.999773,0.699100
2,Roseau,0.946153,1.000000,0.972331


In [22]:
source_breaking_metrics_data = compute_metrics_for_breaking_type(full_data, SOURCE_BREAKING_SUFFIX, tools_analyzed)

source_breaking_metrics_data

,tool,precision,recall,f1
0,Japicmp,0.461936,0.995311,0.631012
1,Revapi,0.462784,0.997091,0.632161
2,Roseau,0.832295,0.994759,0.906304


In [23]:
print(full_data['Ground Truth Global'].value_counts())
print(full_data['Ground Truth Binary'].value_counts())
print(full_data['Ground Truth Source'].value_counts())

True     48625
False    47607
Name: Ground Truth Global, dtype: int64
False    52199
True     44033
Name: Ground Truth Binary, dtype: int64
False    56355
True     39877
Name: Ground Truth Source, dtype: int64
